# 11.24과제

### 1. 사이트에서 동이름까지 크롤링해서 갖고오기

In [1]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [2]:
driver = webdriver.Chrome('./chromedriver')
driver.maximize_window()
driver.get('http://www.opinet.co.kr/searRgSelect.do')
time.sleep(3)

driver.find_element_by_css_selector('.ic_m1').click()
time.sleep(2)

region = driver.find_element_by_xpath('//*[@id="SIGUNGU_NM0"]/option[5]')
dong_1 = driver.find_element_by_xpath('//*[@id="DONG_NM"]/option[9]').text
dong_2 = driver.find_element_by_xpath('//*[@id="DONG_NM"]/option[14]').text
dong_3 = driver.find_element_by_xpath('//*[@id="DONG_NM"]/option[25]').text

dong_list = [dong_1, dong_2, dong_3]

for dong in dong_list:
    region = driver.find_element_by_xpath('//*[@id="SIGUNGU_NM0"]/option[5]').click()
    dongs = driver.find_element_by_id('DONG_NM')
    dongs.send_keys(dong)
    time.sleep(1)

    driver.find_element_by_xpath('//*[@id="glopopd_excel"]/span').click()
    time.sleep(2)

driver.quit()

In [3]:
import pandas as pd
from glob import glob

In [4]:
glob('./station_dong_data/지역*.xls')

['./station_dong_data\\지역_위치별(주유소) (1).xls',
 './station_dong_data\\지역_위치별(주유소) (2).xls',
 './station_dong_data\\지역_위치별(주유소).xls']

In [5]:
df = pd.read_excel('./station_dong_data/지역_위치별(주유소).xls', header=2)
df.head()

,지역,상호,주소,상표,전화번호,셀프여부,고급휘발유,휘발유,경유,실내등유
0,서울특별시,대성석유(주)직영관문주유소,서울 강서구 공항대로 314,GS칼텍스,02-3665-2051,N,1599,1358,1187,1000
1,서울특별시,우장산주유소,서울 강서구 강서로 292 (내발산동),SK에너지,02-3664-7163,N,-,1358,1187,1000


In [6]:
station_files = glob('./station_dong_data/지역*.xls')
xls_raw = []

for file_name in station_files:
    xls = pd.read_excel(file_name, header=2) #header=2라면 위의 2줄은 건너뛰고 3번째줄부터 데이터를 읽는다.
    xls_raw.append(xls)

dong_station = pd.concat(xls_raw, ignore_index=True)
dong_station

,지역,상호,주소,상표,전화번호,셀프여부,고급휘발유,휘발유,경유,실내등유
0,서울특별시,대성석유(주)방화대성주유소,서울 강서구 양천로 176,GS칼텍스,02-3663-7781,Y,1599,1365,1175,-
1,서울특별시,대성석유(주) 마곡대성주유소,서울 강서구 강서로 457,GS칼텍스,02-3664-7781,Y,1599,1365,1185,-
2,서울특별시,화곡역주유소,서울특별시 강서구 강서로 154 (화곡동),알뜰주유소,02-2691-5151,Y,-,1243,1043,-
3,서울특별시,뉴신정주유소,서울 강서구 곰달래로 207(화곡동),알뜰주유소,02-2601-0077,N,-,1243,1043,-
4,서울특별시,목화주유소,서울 강서구 국회대로 251 (화곡동),SK에너지,02-2654-4564,Y,-,1266,1072,-
5,서울특별시,지에스칼텍스㈜ 경인고속주유소,서울 강서구 국회대로 225 (화곡동),GS칼텍스,02-2608-5151,Y,1416,1266,1072,1100
6,서울특별시,신화곡주유소,서울 강서구 국회대로 71,S-OIL,02-2611-3161,Y,-,1315,1115,-
7,서울특별시,화곡현대주유소,서울 강서구 강서로 151 (화곡동),SK에너지,02-2607-1942,Y,1455,1326,1126,-
8,서울특별시,대성석유(주)직영관문주유소,서울 강서구 공항대로 314,GS칼텍스,02-3665-2051,N,1599,1358,1187,1000
9,서울특별시,우장산주유소,서울 강서구 강서로 292 (내발산동),SK에너지,02-3664-7163,N,-,1358,1187,1000


In [7]:
import requests
from urllib.parse import urlparse, quote
import folium

In [8]:
key_fd = open('kakaomapkey.txt', mode='r')
kmap_key = key_fd.read(100)
key_fd.close()

In [9]:
from tqdm import tqdm_notebook

lat_list=[]; lng_list=[]

for i in tqdm_notebook(dong_station.index):
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + quote(dong_station['주소'][i])
    result = requests.get(url, headers={"Authorization": "KakaoAK "+ kmap_key}).json()
    
    lat = float(result['documents'][0]['y'])
    lng = float(result['documents'][0]['x'])
    lat_list.append(lat)
    lng_list.append(lng)

In [10]:
dong_station['lat'] = lat_list
dong_station['lng'] = lng_list
dong_station

,지역,상호,주소,상표,전화번호,셀프여부,고급휘발유,휘발유,경유,실내등유,lat,lng
0,서울특별시,대성석유(주)방화대성주유소,서울 강서구 양천로 176,GS칼텍스,02-3663-7781,Y,1599,1365,1175,-,37.572898,126.823440
1,서울특별시,대성석유(주) 마곡대성주유소,서울 강서구 강서로 457,GS칼텍스,02-3664-7781,Y,1599,1365,1185,-,37.566243,126.840421
2,서울특별시,화곡역주유소,서울특별시 강서구 강서로 154 (화곡동),알뜰주유소,02-2691-5151,Y,-,1243,1043,-,37.540768,126.841472
3,서울특별시,뉴신정주유소,서울 강서구 곰달래로 207(화곡동),알뜰주유소,02-2601-0077,N,-,1243,1043,-,37.531562,126.856035
4,서울특별시,목화주유소,서울 강서구 국회대로 251 (화곡동),SK에너지,02-2654-4564,Y,-,1266,1072,-,37.530055,126.862834
5,서울특별시,지에스칼텍스㈜ 경인고속주유소,서울 강서구 국회대로 225 (화곡동),GS칼텍스,02-2608-5151,Y,1416,1266,1072,1100,37.529645,126.859835
6,서울특별시,신화곡주유소,서울 강서구 국회대로 71,S-OIL,02-2611-3161,Y,-,1315,1115,-,37.527320,126.842879
7,서울특별시,화곡현대주유소,서울 강서구 강서로 151 (화곡동),SK에너지,02-2607-1942,Y,1455,1326,1126,-,37.540242,126.841164
8,서울특별시,대성석유(주)직영관문주유소,서울 강서구 공항대로 314,GS칼텍스,02-3665-2051,N,1599,1358,1187,1000,37.557842,126.842395
9,서울특별시,우장산주유소,서울 강서구 강서로 292 (내발산동),SK에너지,02-3664-7163,N,-,1358,1187,1000,37.551803,126.836681


In [11]:
dong_station = dong_station.sort_values(by='휘발유')
dong_station

,지역,상호,주소,상표,전화번호,셀프여부,고급휘발유,휘발유,경유,실내등유,lat,lng
2,서울특별시,화곡역주유소,서울특별시 강서구 강서로 154 (화곡동),알뜰주유소,02-2691-5151,Y,-,1243,1043,-,37.540768,126.841472
3,서울특별시,뉴신정주유소,서울 강서구 곰달래로 207(화곡동),알뜰주유소,02-2601-0077,N,-,1243,1043,-,37.531562,126.856035
4,서울특별시,목화주유소,서울 강서구 국회대로 251 (화곡동),SK에너지,02-2654-4564,Y,-,1266,1072,-,37.530055,126.862834
5,서울특별시,지에스칼텍스㈜ 경인고속주유소,서울 강서구 국회대로 225 (화곡동),GS칼텍스,02-2608-5151,Y,1416,1266,1072,1100,37.529645,126.859835
6,서울특별시,신화곡주유소,서울 강서구 국회대로 71,S-OIL,02-2611-3161,Y,-,1315,1115,-,37.527320,126.842879
7,서울특별시,화곡현대주유소,서울 강서구 강서로 151 (화곡동),SK에너지,02-2607-1942,Y,1455,1326,1126,-,37.540242,126.841164
8,서울특별시,대성석유(주)직영관문주유소,서울 강서구 공항대로 314,GS칼텍스,02-3665-2051,N,1599,1358,1187,1000,37.557842,126.842395
9,서울특별시,우장산주유소,서울 강서구 강서로 292 (내발산동),SK에너지,02-3664-7163,N,-,1358,1187,1000,37.551803,126.836681
0,서울특별시,대성석유(주)방화대성주유소,서울 강서구 양천로 176,GS칼텍스,02-3663-7781,Y,1599,1365,1175,-,37.572898,126.823440
1,서울특별시,대성석유(주) 마곡대성주유소,서울 강서구 강서로 457,GS칼텍스,02-3664-7781,Y,1599,1365,1185,-,37.566243,126.840421


In [12]:
dong_station['휘발유'] = pd.to_numeric(dong_station['휘발유'])
dong_station

,지역,상호,주소,상표,전화번호,셀프여부,고급휘발유,휘발유,경유,실내등유,lat,lng
2,서울특별시,화곡역주유소,서울특별시 강서구 강서로 154 (화곡동),알뜰주유소,02-2691-5151,Y,-,1243,1043,-,37.540768,126.841472
3,서울특별시,뉴신정주유소,서울 강서구 곰달래로 207(화곡동),알뜰주유소,02-2601-0077,N,-,1243,1043,-,37.531562,126.856035
4,서울특별시,목화주유소,서울 강서구 국회대로 251 (화곡동),SK에너지,02-2654-4564,Y,-,1266,1072,-,37.530055,126.862834
5,서울특별시,지에스칼텍스㈜ 경인고속주유소,서울 강서구 국회대로 225 (화곡동),GS칼텍스,02-2608-5151,Y,1416,1266,1072,1100,37.529645,126.859835
6,서울특별시,신화곡주유소,서울 강서구 국회대로 71,S-OIL,02-2611-3161,Y,-,1315,1115,-,37.527320,126.842879
7,서울특별시,화곡현대주유소,서울 강서구 강서로 151 (화곡동),SK에너지,02-2607-1942,Y,1455,1326,1126,-,37.540242,126.841164
8,서울특별시,대성석유(주)직영관문주유소,서울 강서구 공항대로 314,GS칼텍스,02-3665-2051,N,1599,1358,1187,1000,37.557842,126.842395
9,서울특별시,우장산주유소,서울 강서구 강서로 292 (내발산동),SK에너지,02-3664-7163,N,-,1358,1187,1000,37.551803,126.836681
0,서울특별시,대성석유(주)방화대성주유소,서울 강서구 양천로 176,GS칼텍스,02-3663-7781,Y,1599,1365,1175,-,37.572898,126.823440
1,서울특별시,대성석유(주) 마곡대성주유소,서울 강서구 강서로 457,GS칼텍스,02-3664-7781,Y,1599,1365,1185,-,37.566243,126.840421


In [13]:
map = folium.Map(location=[dong_station['lat'].mean(), dong_station['lng'].mean()], zoom_start=13)

for i in dong_station.index:
    frame = f"{dong_station['상호'][i]}"
    popup = folium.Popup(frame, max_width=150)
    if dong_station['휘발유'][i] == min(dong_station['휘발유']):
        folium.CircleMarker([dong_station['lat'][i], dong_station['lng'][i]],
                            radius=15,
                            color='#CD3181',
                            fill_color='#CD3181',
                            tooltip=(f"{dong_station['휘발유'][i]}원"),
                            popup=popup
                            ).add_to(map)
    else:
        folium.CircleMarker([dong_station['lat'][i], dong_station['lng'][i]],
                            radius=15,
                            color='#4B89DC',
                            fill_color='#4B89DC',
                            tooltip=(f"{dong_station['휘발유'][i]}원"),
                            popup=popup
                            ).add_to(map)
map

### 2. 이름을 키워드로 카카오맵에서 검색 후, 서울 전체엑셀에서 inner join하기

In [14]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import requests
from urllib.parse import urlparse, quote
import folium
import pandas as pd
from glob import glob

In [15]:
station_list = ['우장산주유소', '대성석유(주)직영관문주유소	', '화곡현대주유소', '신화곡주유소', '지에스칼텍스㈜ 경인고속주유소', '목화주유소', '뉴신정주유소', '화곡역주유소', '대성석유(주) 마곡대성주유소', '대성석유(주)방화대성주유소']

In [16]:
key_fd = open('kakaomapkey.txt', mode='r')
kmap_key = key_fd.read(100)
key_fd.close()

In [17]:
addr = '우장산주유소'
url = 'https://dapi.kakao.com/v2/local/search/keyword.json?query=' + quote(addr)
result = requests.get(url,headers={'Authorization': "KakaoAK "+kmap_key}).json()
result['documents'][0]

{'address_name': '서울 강서구 내발산동 702-9',
 'category_group_code': 'OL7',
 'category_group_name': '주유소,충전소',
 'category_name': '교통,수송 > 자동차 > 주유,가스 > 주유소 > SK주유소',
 'distance': '',
 'id': '27163582',
 'phone': '02-3664-7163',
 'place_name': '우장산주유소',
 'place_url': 'http://place.map.kakao.com/27163582',
 'road_address_name': '서울 강서구 강서로 292',
 'x': '126.836581578392',
 'y': '37.5516105340577'}

In [18]:
name_list=[]; tel_list=[]; addr_list=[]; lat_list=[]; lng_list=[]

for station in station_list:
    url = 'https://dapi.kakao.com/v2/local/search/keyword.json?query=' + quote(station)
    result = requests.get(url,headers={'Authorization': "KakaoAK "+kmap_key}).json()
    name_list.append(result['documents'][0]['place_name'])
    tel_list.append(result['documents'][0]['phone'])
    addr_list.append(result['documents'][0]['road_address_name'])
    lat_list.append(float(result['documents'][0]['y']))
    lng_list.append(float(result['documents'][0]['x']))

In [19]:
station_data = pd.DataFrame({
    'place_name':name_list,
    'tel':tel_list,
    'address':addr_list,
    'lat':lat_list,
    'lng':lng_list
})
station_data

,place_name,tel,address,lat,lng
0,우장산주유소,02-3664-7163,서울 강서구 강서로 292,37.551611,126.836582
1,대성석유직영 관문주유소,02-3665-2051,서울 강서구 공항대로 314,37.557875,126.842367
2,화곡현대주유소,02-2607-1942,서울 강서구 강서로 151,37.540264,126.841200
3,신화곡주유소,02-2611-3161,서울 강서구 국회대로 71,37.527260,126.842889
4,GS칼텍스 경인고속주유소,02-2608-5151,서울 강서구 국회대로 225,37.529624,126.859795
5,목화주유소,02-2654-4564,서울 강서구 국회대로 251,37.530020,126.863062
6,뉴신정주유소,02-2601-0077,서울 강서구 곰달래로 207,37.531550,126.856042
7,화곡역주유소,02-2691-5151,서울 강서구 강서로 154,37.540625,126.841525
8,대성석유 마곡대성주유소,02-3664-7781,서울 강서구 강서로 457,37.566279,126.840412
9,대성석유방화대성주유소,02-3663-7781,서울 강서구 양천로 176,37.572898,126.823436


In [20]:
driver = webdriver.Chrome('./chromedriver')
driver.maximize_window()
driver.get('http://www.opinet.co.kr/searRgSelect.do')
time.sleep(3)

driver.find_element_by_css_selector('.ic_m1').click()
time.sleep(2)

region = driver.find_element_by_xpath('//*[@id="SIGUNGU_NM0"]')
gu_list = region.find_elements_by_tag_name('option')

gu_names = [gu.get_attribute('value') for gu in gu_list] # 파이썬다운코드
del gu_names[0]

for gu in gu_names:
    region = driver.find_element_by_id('SIGUNGU_NM0')
    region.send_keys(gu)
    time.sleep(1)

    driver.find_element_by_xpath('//*[@id="glopopd_excel"]/span').click()
    time.sleep(2)

driver.quit()

In [21]:
station_files = glob('./1124_station_data/지역*.xls')
tmp_raw = []

for file_name in station_files:
    tmp = pd.read_excel(file_name, header=2)
    tmp_raw.append(tmp)

station_raw = pd.concat(tmp_raw)
station_raw

,지역,상호,주소,상표,전화번호,셀프여부,고급휘발유,휘발유,경유,실내등유
0,서울특별시,구천면주유소,서울 강동구 구천면로 357 (암사동),현대오일뱅크,02-441-0536,N,-,1374,1187,-
1,서울특별시,지에스칼텍스㈜ 신월주유소,서울 강동구 양재대로 1323 (성내동),GS칼텍스,02-475-2600,N,1585,1385,1188,1000
2,서울특별시,현대오일뱅크㈜직영 고속셀프주유소,서울 강동구 천호대로 1246,현대오일뱅크,02-487-2030,Y,1598,1386,1188,895
3,서울특별시,방아다리주유소,서울 강동구 동남로 811 (명일동),SK에너지,02-442-5145,Y,-,1395,1195,975
4,서울특별시,현대오일뱅크㈜직영 명일셀프주유소,서울 강동구 고덕로 168 (명일동),현대오일뱅크,02-3428-1739,Y,1625,1395,1198,-
...,...,...,...,...,...,...,...,...,...,...
35,서울특별시,(주)소모에너지엔테크놀러지 쎈트럴주유소,서울 강남구 삼성로 335,GS칼텍스,02-565-0267,N,2037,1883,1659,1100
36,서울특별시,(주)만정에너지,서울 강남구 봉은사로 433 (삼성동),GS칼텍스,02-518-5141,N,2079,1913,1689,1359
37,서울특별시,뉴서울(강남),서울 강남구 언주로 716 (논현동),SK에너지,02-518-5631,N,2330,2025,1865,1260
38,서울특별시,동우주유소,서울특별시 강남구 봉은사로 311 (논현동),SK에너지,02-542-6726,N,-,-,-,-


In [22]:
df = pd.merge(station_data, station_raw, left_on='tel', right_on='전화번호', how='inner')
df = df[['place_name', 'address', 'tel', '상표', '휘발유', 'lat', 'lng']]
df = df.sort_values(by='휘발유')
df

,place_name,address,tel,상표,휘발유,lat,lng
6,뉴신정주유소,서울 강서구 곰달래로 207,02-2601-0077,알뜰주유소,1243,37.531550,126.856042
7,화곡역주유소,서울 강서구 강서로 154,02-2691-5151,알뜰주유소,1243,37.540625,126.841525
4,GS칼텍스 경인고속주유소,서울 강서구 국회대로 225,02-2608-5151,GS칼텍스,1266,37.529624,126.859795
5,목화주유소,서울 강서구 국회대로 251,02-2654-4564,SK에너지,1266,37.530020,126.863062
3,신화곡주유소,서울 강서구 국회대로 71,02-2611-3161,S-OIL,1315,37.527260,126.842889
2,화곡현대주유소,서울 강서구 강서로 151,02-2607-1942,SK에너지,1326,37.540264,126.841200
0,우장산주유소,서울 강서구 강서로 292,02-3664-7163,SK에너지,1358,37.551611,126.836582
1,대성석유직영 관문주유소,서울 강서구 공항대로 314,02-3665-2051,GS칼텍스,1358,37.557875,126.842367
8,대성석유 마곡대성주유소,서울 강서구 강서로 457,02-3664-7781,GS칼텍스,1365,37.566279,126.840412
9,대성석유방화대성주유소,서울 강서구 양천로 176,02-3663-7781,GS칼텍스,1365,37.572898,126.823436


In [23]:
map = folium.Map(location=[df['lat'].mean(), df['lng'].mean()], zoom_start=13)

for i in df.index:
    frame = f"{df['place_name'][i]}"
    popup = folium.Popup(frame, max_width=150)
    if df['휘발유'][i] == min(df['휘발유']):
        folium.Marker(
            location = [df['lat'][i], df['lng'][i]],
            tooltip=(f"{df['휘발유'][i]} 원"),
            popup=popup,
            icon=folium.Icon(color='red', icon='thumbs-up')
            ).add_to(map)
    else:
        folium.Marker(
            location = [df['lat'][i], df['lng'][i]],
            tooltip=(f"{df['휘발유'][i]} 원"),
            popup=popup,
            icon=folium.Icon(color='blue', icon='tint')
            ).add_to(map)
map

### 3. 사이트에서 키워드(주유소명)으로 검색하기

In [24]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import requests
from urllib.parse import urlparse, quote
import folium
import pandas as pd
from glob import glob

In [25]:
driver = webdriver.Chrome('./chromedriver')
driver.maximize_window()
driver.get('http://www.opinet.co.kr/searRgSelect.do')
time.sleep(3)

driver.find_element_by_css_selector('.ic_m1').click()
time.sleep(2)
driver.find_element_by_xpath('//*[@id="SEARCH_MOD4"]').click()

station_list = ['우장산주유소', '직영관문주유소', '화곡현대주유소', '신화곡주유소', '경인고속주유소', '목화주유소', '뉴신정주유소', '화곡역주유소', '마곡대성주유소', '방화대성주유소']
gasoline_list = []; diesel_list = []
for station in station_list:
    search = driver.find_element_by_xpath('//*[@id="OS_NM"]')
    time.sleep(1)
    search.clear()
    time.sleep(1)
    search.send_keys(station)
    search.submit()
    time.sleep(1)
    gasoline = driver.find_element_by_xpath('//*[@id="body1"]/tr/td[2]').text
    diesel = driver.find_element_by_xpath('//*[@id="body1"]/tr/td[3]/font').text
    time.sleep(1)
    gasoline_list.append(int(gasoline))
    diesel_list.append(int(diesel))

driver.quit()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="SEARCH_MOD4"]"}
  (Session info: chrome=87.0.4280.66)


In [28]:
key_fd = open('kakaomapkey.txt', mode='r')
kmap_key = key_fd.read(100)
key_fd.close()

In [29]:
name_list=[]; tel_list=[]; addr_list=[]; lat_list=[]; lng_list=[]

for station in station_list:
    url = 'https://dapi.kakao.com/v2/local/search/keyword.json?query=' + quote(station)
    result = requests.get(url,headers={'Authorization': "KakaoAK "+kmap_key}).json()
    name_list.append(result['documents'][0]['place_name'])
    tel_list.append(result['documents'][0]['phone'])
    addr_list.append(result['documents'][0]['road_address_name'])
    lat_list.append(float(result['documents'][0]['y']))
    lng_list.append(float(result['documents'][0]['x']))

In [30]:
station_data = pd.DataFrame({
    'station_name':station_list,
    'tel':tel_list,
    'address':addr_list,
    'gasoline_price':gasoline_list,
    'diesel_price':diesel_list,
    'lat':lat_list,
    'lng':lng_list
})
station_data

,station_name,tel,address,gasoline_price,diesel_price,lat,lng
0,우장산주유소,02-3664-7163,서울 강서구 강서로 292,1358,1187,37.551611,126.836582
1,직영관문주유소,02-3665-2051,서울 강서구 공항대로 314,1358,1187,37.557875,126.842367
2,화곡현대주유소,02-2607-1942,서울 강서구 강서로 151,1326,1126,37.540264,126.841200
3,신화곡주유소,02-2611-3161,서울 강서구 국회대로 71,1315,1115,37.527260,126.842889
4,경인고속주유소,02-2608-5151,서울 강서구 국회대로 225,1266,1072,37.529624,126.859795
5,목화주유소,02-2654-4564,서울 강서구 국회대로 251,1266,1072,37.530020,126.863062
6,뉴신정주유소,02-2601-0077,서울 강서구 곰달래로 207,1243,1043,37.531550,126.856042
7,화곡역주유소,02-2691-5151,서울 강서구 강서로 154,1243,1043,37.540625,126.841525
8,마곡대성주유소,02-3664-7781,서울 강서구 강서로 457,1365,1185,37.566279,126.840412
9,방화대성주유소,02-3663-7781,서울 강서구 양천로 176,1365,1175,37.572898,126.823436


In [31]:
map = folium.Map(location=[station_data['lat'].mean(), station_data['lng'].mean()], zoom_start=13)

for i in station_data.index:
    frame = f"{station_data['station_name'][i]}"
    popup = folium.Popup(frame, max_width=150)
    if station_data['gasoline_price'][i] == min(station_data['gasoline_price']):
        folium.Marker(
            location = [station_data['lat'][i], station_data['lng'][i]],
            tooltip=(f"휘발유:{station_data['gasoline_price'][i]}원, 가솔린:{station_data['diesel_price'][i]}원"),
            popup=popup,
            icon=folium.Icon(color='red', icon='thumbs-up')
            ).add_to(map)
    else:
        folium.Marker(
            location = [station_data['lat'][i], station_data['lng'][i]],
            tooltip=(f"휘발유:{station_data['gasoline_price'][i]}원, 가솔린:{station_data['diesel_price'][i]}원"),
            popup=popup,
            icon=folium.Icon(color='blue', icon='tint')
            ).add_to(map)
map

### 4. opinet에서 도로명주소까지 갖고와서 카카오맵에서 주소로 위도,경도 갖고오기

In [35]:
driver = webdriver.Chrome('./chromedriver')
driver.maximize_window()
driver.get('http://www.opinet.co.kr/searRgSelect.do')
time.sleep(2)

driver.find_element_by_css_selector('.ic_m1').click()
time.sleep(2)
driver.find_element_by_xpath('//*[@id="SEARCH_MOD4"]').click()
time.sleep(1)

station_list = ['우장산주유소', '직영관문주유소', '화곡현대주유소', '신화곡주유소', '경인고속주유소', '목화주유소', '뉴신정주유소', '화곡역주유소', '마곡대성주유소', '방화대성주유소']
gasoline_list = []; diesel_list = []; addr_list=[]; tel_list=[]
for station in station_list:
    search = driver.find_element_by_xpath('//*[@id="OS_NM"]')
    time.sleep(1)
    search.clear()
    time.sleep(2)
    search.send_keys(station)
    time.sleep(1)
    search.submit()
    time.sleep(1)
    driver.find_element_by_xpath('//*[@id="body1"]/tr/td[1]/a').click()
    map_move = driver.find_element_by_xpath('//*[@id="map_div"]/div[4]/div[2]/div[2]/div/div[3]').click()
    time.sleep(1)
    tel = driver.find_element_by_xpath('//*[@id="phn_no"]').text
    addr = driver.find_element_by_xpath('//*[@id="rd_addr"]').text
    gasoline = driver.find_element_by_xpath('//*[@id="body1"]/tr/td[2]').text
    diesel = driver.find_element_by_xpath('//*[@id="body1"]/tr/td[3]/font').text
    time.sleep(10)

    addr_list.append(addr)
    tel_list.append(tel)
    gasoline_list.append(int(gasoline))
    diesel_list.append(int(diesel))

driver.quit()

In [36]:
df2 = pd.DataFrame({
    'station_name':station_list,
    'address':addr_list,
    'tel':tel_list,
    'gasoline':gasoline_list,
    'diesel':diesel_list
})
df2

,station_name,address,tel,gasoline,diesel
0,우장산주유소,서울 강서구 강서로 292 (내발산동),02-3664-7163,1358,1187
1,직영관문주유소,서울 강서구 공항대로 314,02-3665-2051,1358,1187
2,화곡현대주유소,서울 강서구 강서로 151 (화곡동),02-2607-1942,1326,1126
3,신화곡주유소,서울 강서구 국회대로 71,02-2611-3161,1315,1115
4,경인고속주유소,서울 강서구 국회대로 225 (화곡동),02-2608-5151,1266,1072
5,목화주유소,서울 강서구 국회대로 251 (화곡동),02-2654-4564,1266,1072
6,뉴신정주유소,서울 강서구 곰달래로 207(화곡동),02-2601-0077,1243,1043
7,화곡역주유소,서울특별시 강서구 강서로 154 (화곡동),02-2691-5151,1243,1043
8,마곡대성주유소,서울 강서구 강서로 457,02-3664-7781,1365,1185
9,방화대성주유소,서울 강서구 양천로 176,02-3663-7781,1365,1175


In [37]:
lat_list = []; lng_list = []
for i in df.index:
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + quote(df['address'][i])
    result = requests.get(url,
            headers={"Authorization": "KakaoAK "+kmap_key}).json()
    lat = float(result['documents'][0]['y'])
    lng = float(result['documents'][0]['x'])
    lat_list.append(lat)
    lng_list.append(lng)

In [40]:
df2['lat'] = lat_list; df2['lng'] = lng_list
df2

,station_name,address,tel,gasoline,diesel,lat,lng
0,우장산주유소,서울 강서구 강서로 292 (내발산동),02-3664-7163,1358,1187,37.531562,126.856035
1,직영관문주유소,서울 강서구 공항대로 314,02-3665-2051,1358,1187,37.540768,126.841472
2,화곡현대주유소,서울 강서구 강서로 151 (화곡동),02-2607-1942,1326,1126,37.529645,126.859835
3,신화곡주유소,서울 강서구 국회대로 71,02-2611-3161,1315,1115,37.530055,126.862834
4,경인고속주유소,서울 강서구 국회대로 225 (화곡동),02-2608-5151,1266,1072,37.527320,126.842879
5,목화주유소,서울 강서구 국회대로 251 (화곡동),02-2654-4564,1266,1072,37.540242,126.841164
6,뉴신정주유소,서울 강서구 곰달래로 207(화곡동),02-2601-0077,1243,1043,37.551803,126.836681
7,화곡역주유소,서울특별시 강서구 강서로 154 (화곡동),02-2691-5151,1243,1043,37.557842,126.842395
8,마곡대성주유소,서울 강서구 강서로 457,02-3664-7781,1365,1185,37.566243,126.840421
9,방화대성주유소,서울 강서구 양천로 176,02-3663-7781,1365,1175,37.572898,126.823440


In [41]:
map = folium.Map(location=[df2['lat'].mean(), df2['lng'].mean()], zoom_start=13)

for i in df2.index:
    frame = f"{df2['station_name'][i]}"
    popup = folium.Popup(frame, max_width=130)
    if df2['gasoline'][i] == min(df2['gasoline']):
        folium.Marker(
            location = [df2['lat'][i], df2['lng'][i]],
            tooltip=(f"휘발유:{df2['gasoline'][i]}원, 가솔린:{df2['diesel'][i]}원"),
            popup=popup,
            icon=folium.Icon(color='red', icon='thumbs-up')
            ).add_to(map)
    else:
        folium.Marker(
            location = [df2['lat'][i], df2['lng'][i]],
            tooltip=(f"휘발유:{df2['gasoline'][i]}원, 가솔린:{df2['diesel'][i]}원"),
            popup=popup,
            icon=folium.Icon(color='blue', icon='tint')
            ).add_to(map)
map